In [ ]:
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.agents import AssistantAgent
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_agentchat.conditions import MaxMessageTermination, TextMentionTermination
from autogen_agentchat.ui import Console




In [ ]:
model = OpenAIChatCompletionClient(model="gpt-5-mini")
clarity_agent = AssistantAgent(
    "ClarityAgent",
    model_client=model,
    system_message="""You are an expert editor focused on clarity and simplicity.
    Your job is to eliminate ambiguity, redundancy, and make every sentence crisp and clear.
    Don't worry about persuasion or tone -- just make the message easy to read and understand."""
)
tone_agent=AssistantAgent(
    "ToneAgent",
    model_client=model,
    system_message="""You are a communication coach focused on emotional tone and professionalism.
    Your job is to make the email sound warm, confident, and human -- while staying professional and appropriate for the audience.
    Improve the emotional resonance, polish the phrasing, and adjust any words that may come off as stiff, cold, or overly casual."""
)
persuasion_agent=AssistantAgent(
    "PersuasionAgent",
    model_client=model,
    system_message="""You are a persuasion expert trained in marketing, behavioral psychology, and copywriting.
    Your job is to enhance the email's persuasive power: improve call to action, structure arguments, emphasize benefits.
    Remove weak or passive language."""
)
synthesizer_agent=AssistantAgent(
    "SynthesizerAgent",
    model_client=model,
    system_message=""
)
critic_agent=AssistantAgent(
    "CriticAgent",
    model_client=model,
    system_message="""You are an email quality evaluator. 
    Your job is to perform a final review of the synthesized email and determine if it meets professional standards.
    Review the email for:
    
    """
)